In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from lstm import F1PredictionModel, load_model_with_preprocessor
from race_simulator import Race, RaceSimulator, Driver
from features import RaceFeatures

In [2]:

# Load the trained model
model_path = 'models/f1_prediction_model.pth'
model, preprocessor = load_model_with_preprocessor(model_path)

# Create Race instance
safety_car_periods = [(20, 22)]  # Safety car from lap 20 to 22

# Create an instance of the Race
race = Race(
    race_id=900,
    circuit_id=1,
    total_laps=50,
    weather_conditions={},  # Add weather conditions if needed
    safety_car_periods=safety_car_periods
)

# Load driver attributes
drivers_df = pd.read_csv('drivers_attributes.csv')

# Initialize drivers and add to race
for idx, row in drivers_df.iterrows():
    driver = Driver(
        driver_id=row['driverId'],
        name=row.get('driver_name', f'Driver {row["driverId"]}'),
        static_features=row[['driver_overall_skill', 'driver_circuit_skill', 'driver_consistency',
                             'driver_reliability', 'driver_aggression', 'driver_risk_taking']].values,
        initial_dynamic_features={
            'tire_age': 0,
            'fuel_load': 100.0,
            'track_position': idx + 1,  # Starting position
            'track_temp': 35.0,
            'air_temp': 25.0,
            'humidity': 50.0,
            'TrackStatus': 1,
            'is_pit_lap': 0,
            'safety_car': 0,
            'tire_compound': 1  # Starting on Hard compound
        },
        start_position=idx + 1,
        pit_strategy=[(25, 2)],  # Example pit strategy
        starting_compound=1
    )
    # Scale static features
    driver.static_features = preprocessor.static_scaler.transform([driver.static_features]).flatten()
    # Initialize sequence
    driver.sequence = np.zeros((3, len(preprocessor.race_features.dynamic_features) + 1))
    race.drivers.append(driver)

# Simulate the race
simulator = RaceSimulator(model, preprocessor)
race_lap_data = simulator.simulate_race(race)

# Analyze results
def plot_race_positions(race):
    plt.figure(figsize=(12, 6))
    
    for driver in race.drivers:
        positions = race.lap_data[driver.driver_id]['positions']
        plt.plot(range(1, race.total_laps + 1), positions, label=driver.name)
    
    plt.gca().invert_yaxis()  # Invert y-axis so that position 1 is at the top
    plt.xlabel('Lap')
    plt.ylabel('Position')
    plt.title('Race Simulation: Driver Positions Over Laps')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_lap_times(race):
    plt.figure(figsize=(12, 6))
    
    for driver in race.drivers:
        lap_times = race.lap_data[driver.driver_id]['lap_times']
        plt.plot(range(1, race.total_laps + 1), lap_times, label=driver.name)
    
    plt.xlabel('Lap')
    plt.ylabel('Lap Time (ms)')
    plt.title('Race Simulation: Driver Lap Times')
    plt.legend()
    plt.grid(True)
    plt.show()

def create_lap_times_dataframe(race) -> pd.DataFrame:
    data = {'Lap': []}
    total_laps = race.total_laps
    for driver in race.drivers:
        data[driver.name] = race.lap_data[driver.driver_id]['lap_times']
    data['Lap'] = list(range(1, total_laps + 1))
    lap_times_df = pd.DataFrame(data)
    return lap_times_df

def create_lap_times_with_inputs_dataframe(race, race_features) -> pd.DataFrame:
    records = []
    for driver in race.drivers:
        lap_times = race.lap_data[driver.driver_id]['lap_times']
        positions = race.lap_data[driver.driver_id]['positions']
        inputs_list = race.lap_data[driver.driver_id]['inputs']
        for lap_index, (lap_time, position, inputs) in enumerate(zip(lap_times, positions, inputs_list)):
            record = {
                'Lap': lap_index + 1,
                'Driver': driver.name,
                'LapTime': lap_time,
                'Position': position,
            }
            # Flatten static and dynamic features
            for i, feature_name in enumerate(race_features.static_features):
                record[feature_name] = inputs['static_features'][i]
            for feature_name, value in inputs['dynamic_features'].items():
                record[feature_name] = value
            records.append(record)
    lap_times_df = pd.DataFrame(records)
    return lap_times_df


/Users/I551659/Documents/GitHub/IE650-RAMP/ie500-data-mining-group7/lap_simulation/lstm.py:178: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, m

FileNotFoundError: [Errno 2] No such file or directory: 'drivers_attributes.csv'

In [ ]:
simulator = RaceSimulator(model, preprocessor)
race_lap_data = simulator.simulate_race(race)